In [534]:
from selenium import webdriver
from PIL import Image
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains

In [535]:
options = webdriver.ChromeOptions() 
options.add_argument("download.default_directory=/Users/joel/library_/coronavirus")

driver = webdriver.Chrome('/Users/joel/selenium_drivers/chromedriver 3',
                          chrome_options=options)

/Users/joel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [539]:
def location(tweet):
    location = tweet.location
    size = tweet.size
    driver.save_screenshot('/Users/joel/library_/extremism/extremism-{}.png'.format(count))
    x = location['x']
    y = location['y']
    width = location['x']+size['width']
    height = location['y']+size['height']

    return x, y, width, height

def crop_screenshot(count, x, y, width, height, path):    
    im = Image.open('{}-{}.png'.format(path, count))

    im = im.crop((int(x), int(y), int(width), int(height)))
    im.save('{}-{}.png'.format(path, count))

def find_tweet(driver, url):
    driver.get(url)
    sleep(1)
    driver.find_element_by_tag_name('article').click()
    tweets = driver.find_elements_by_tag_name('article')
    username = tweets[0].text.split('\n')[0]
    x, y, width, height = location(tweets[0])
    
    return tweets, username, x, y, width, height

def thread_screenshot(tweet, count, x, y, width, height, username, path):
    try:
        tweet.click()
    except:
        driver.execute_script("scrollBy(0,250);")
        sleep(1)
        driver.execute_script("scrollBy(0,-250);")
        sleep(1)
        driver.execute_script("scrollBy(0,250);")
        tweet.click()
    sleep(2)
    driver.save_screenshot('{}-{}.png'.format(path, count))
    crop_screenshot(count, x, y, width, height, path)

    try:
        found = driver.find_elements_by_tag_name('article')
        tweet_list = [element for element in found if element.text.split('\n')[0] == username]
    except:
        sleep(1)
        found = driver.find_elements_by_tag_name('article')
        tweet_list = [element for element in found if element.text.split('\n')[0] == username]
    
    for tweet_ in tweet_list:
        split = tweet_.text.split('\n')
        if 'Replying to ' in split:
            thread_screenshot(tweet_, count+1, x, y, width, height, username, path)
            break

def twitter_thread(driver, url, path):
    
    tweets, username, x, y, width, height = find_tweet(driver, url)
    
    thread_screenshot(tweets[0], 0, x, y, width, height, username, path)

In [ ]:
url = 'https://twitter.com/ScottGottliebMD/status/1236473220783636481?s=20'
path = '/Users/joel/library_/coronavirus/@scottgrottliebmd'

In [542]:
twitter_thread(driver, url, path)